In [ ]:
import numpy as np
import matplotlib as plt
from scipy.integrate import quad
from scipy.optimize import fsolve

# Fuel assembly (FA) representative of average core conditions
P_FA = 6.641 #[MW] Fuel assembly thermal power
N_FP = 284 #[pin] Number of fuel pins in the fuel assembly
q_l0 = 26.03 #[kW/m] Linear heat rate at fuel midplane, q'(0)
P_FP = P_FA/N_FP #[MW/pin] Average thermal power per fuel pin
H_act = 1100 #[mm] Active length
P_FP = P_FA/N_FP #[MW/pin]
Int_obj = (P_FP*1000/(2*q_l0)) #[m]

print(P_FP*1000)
print(Int_obj)

# Generate multiple values of H_ext for different n
def calculate_H_ext(H_act, n_values):
    H_ext_values = []
    for n in n_values:
        H_ext = H_act / (2 * n)
        H_ext_values.append(H_ext)
    return H_ext_values

# Define n values
n_values = range(250, 400)  # Starts from 1 to avoid division by zero

# Function to integrate cos(pi*z/H_ext) from 0 to H_t/2
def integrand(z, H_ext):
    return np.cos(np.pi * z / H_ext)

# Calculate the integral
def calculate_integral(H_ext):
    H_t = H_act + H_ext  # Total length H_t = H_act + H_ext
    upper_limit = H_t / 2
    integral_result, _ = quad(integrand, 0, upper_limit, args=(H_ext,))
    return integral_result

# Calculate H_ext values
H_ext_values = calculate_H_ext(H_act, n_values)

# Loop to verify extrapolated length and calculate integral for each H_ext
for H_ext in H_ext_values:
    z = (H_act + H_ext) / 2
    a = np.cos(np.pi * z / H_ext)

    # Calculate the integral
    integral_value = calculate_integral(H_ext)

    # Print the results
    print(f"For H_ext = {H_ext:.2f} mm, z = {z:.2f} mm, cos(pi*z/H_ext) = {a:.2f}, Integral = {integral_value:.2f}")





23.383802816901408
0.4491702423530812
For H_ext = 2.20 mm, z = 551.10 mm, cos(pi*z/H_ext) = 0.00, Integral = 0.70
For H_ext = 2.19 mm, z = 551.10 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.70
For H_ext = 2.18 mm, z = 551.09 mm, cos(pi*z/H_ext) = -0.00, Integral = 0.69
For H_ext = 2.17 mm, z = 551.09 mm, cos(pi*z/H_ext) = 0.00, Integral = -0.69
For H_ext = 2.17 mm, z = 551.08 mm, cos(pi*z/H_ext) = 0.00, Integral = 0.69
For H_ext = 2.16 mm, z = 551.08 mm, cos(pi*z/H_ext) = 0.00, Integral = -0.69
For H_ext = 2.15 mm, z = 551.07 mm, cos(pi*z/H_ext) = -0.00, Integral = 0.68
For H_ext = 2.14 mm, z = 551.07 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.68
For H_ext = 2.13 mm, z = 551.07 mm, cos(pi*z/H_ext) = 0.00, Integral = 0.68
For H_ext = 2.12 mm, z = 551.06 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.68
For H_ext = 2.12 mm, z = 551.06 mm, cos(pi*z/H_ext) = 0.00, Integral = -1.72
For H_ext = 2.11 mm, z = 551.05 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.67
For H_ext = 2.10 mm, z = 551.05 mm, c

<ipython-input-33-726a88b8cc69>:37: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral_result, _ = quad(integrand, 0, upper_limit, args=(H_ext,))
<ipython-input-33-726a88b8cc69>:37: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral_result, _ = quad(integrand, 0, upper_limit, args=(H_ext,))



For H_ext = 1.64 mm, z = 550.82 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.52
For H_ext = 1.64 mm, z = 550.82 mm, cos(pi*z/H_ext) = -0.00, Integral = 0.52
For H_ext = 1.63 mm, z = 550.82 mm, cos(pi*z/H_ext) = 0.00, Integral = 3.40
For H_ext = 1.63 mm, z = 550.81 mm, cos(pi*z/H_ext) = 0.00, Integral = -2.35
For H_ext = 1.62 mm, z = 550.81 mm, cos(pi*z/H_ext) = 0.00, Integral = -0.52
For H_ext = 1.62 mm, z = 550.81 mm, cos(pi*z/H_ext) = 0.00, Integral = 0.52
For H_ext = 1.61 mm, z = 550.81 mm, cos(pi*z/H_ext) = 0.00, Integral = -0.52
For H_ext = 1.61 mm, z = 550.80 mm, cos(pi*z/H_ext) = -0.00, Integral = 0.53
For H_ext = 1.60 mm, z = 550.80 mm, cos(pi*z/H_ext) = -0.00, Integral = -2.34
For H_ext = 1.60 mm, z = 550.80 mm, cos(pi*z/H_ext) = 0.00, Integral = 0.51
For H_ext = 1.59 mm, z = 550.80 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.49
For H_ext = 1.59 mm, z = 550.79 mm, cos(pi*z/H_ext) = -0.00, Integral = 0.48
For H_ext = 1.59 mm, z = 550.79 mm, cos(pi*z/H_ext) = -0.00, Integral = -0.

In [ ]:
# Define the function to compute the integral for a given H_ext
def compute_integral(H_ext):
    # Perform the integral of q_l from 0 to (H_act/2 + H_ext)
    result, _ = quad(q_l, 0, H_act/2 + H_ext, args=(H_ext,))
    return result



# Define constants
int_q_l = (P_FP*1000)/2 #[kW] Integral from z=0 to z=H_act/2 + H_ext is equal to the half of average power of 1 fuel pin
print('Average thermal power per half fuel pin:', int_q_l, '[kW]')


# Define the function q_l(z)
def q_l(z, H_ext):
    return q_l0 * np.cos(np.pi * z / H_ext)


# Iterate through each H_ext and check if it satisfies the integral condition
for n, H_ext in zip(n_values, H_ext_values):
    integral_value = compute_integral(H_ext)
    print(f"n={n}, H_ext={H_ext:.2f}, Integral={integral_value:.2f}")

    # Check if the integral value matches the target integral
    if np.isclose(integral_value, int_q_l, atol=0.01):
        print(f"Found the correct H_ext: {H_ext} for n={n}")
        break


# Define the range of n values to check (n = 0, 1, 2, 3, ...)
n_values = range(1,10)  # You can adjust the number of solutions by changing this range
H_ext_values = calculate_H_ext(H_act, n_values)

# Printing the results in a formatted manner
for n, H_ext in zip(n_values, H_ext_values):
    print(f"For n = {n}: H_ext = {H_ext:.2f} mm")